<a href="https://colab.research.google.com/github/Vishal123-max/Fist-Java-Code/blob/main/NLP_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP Assignment - 2

**Dataset:** FinancialPhraseBank

**Group:** Group 76

#### Team Members
| Name | Roll No. | Contributions |
|------|----------|---------------|
| Taniya Yadav | 2024DA04119 | 100% |
| SRUTILEKHA DAS | 2022DC04094 | 100% |
| Vishal Yadav | 2024DA04134 | 100% |

**1. Introduction**

We are working with the FinancialPhraseBank dataset, which contains financial news headlines labeled with sentiments (positive, negative, neutral).

Our tasks include:
- Exploratory Data Analysis (EDA)
- Preprocessing
- Topic Modeling with LDA (10 topics)
- Coherence Score
- Visualization of topics
- Dependency Parsing of two sentences


**2. Load and Explore Dataset (EDA)**

In [2]:
import pandas as pd

# Load first 2000 rows
df = pd.read_csv("all-data.csv", nrows=2000, names=["Sentiment", "News Headline"], encoding='latin1')

# Quick look
print(df.head())
print("\nSentiment distribution:\n", df['Sentiment'].value_counts())
print("\nMissing values:\n", df.isnull().sum())

  Sentiment                                      News Headline
0   neutral  According to Gran , the company has no plans t...
1   neutral  Technopolis plans to develop in stages an area...
2  negative  The international electronic industry company ...
3  positive  With the new production plant the company woul...
4  positive  According to the company 's updated strategy f...

Sentiment distribution:
 Sentiment
positive    1031
neutral      902
negative      67
Name: count, dtype: int64

Missing values:
 Sentiment        0
News Headline    0
dtype: int64


**Explanation:**
- We load only 2000 rows.
- Check sentiment distribution.
- Verify missing values.


**3. Preprocessing**

In [3]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # remove punctuation/numbers
    tokens = text.split()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return tokens

df['tokens'] = df['News Headline'].apply(preprocess)
df[['News Headline','tokens']].head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


,News Headline,tokens
0,"According to Gran , the company has no plans t...","[according, gran, company, plan, move, product..."
1,Technopolis plans to develop in stages an area...,"[technopolis, plan, develop, stage, area, less..."
2,The international electronic industry company ...,"[international, electronic, industry, company,..."
3,With the new production plant the company woul...,"[new, production, plant, company, would, incre..."
4,According to the company 's updated strategy f...,"[according, company, updated, strategy, year, ..."


**Explanation:**
- Lowercasing, punctuation removal.
- Tokenization.
- Stopword removal.
- Lemmatization.


**4. Topic Modeling with LDA (10 Topics)**

In [4]:
!pip install gensim

from gensim import corpora, models

# Create dictionary and corpus
dictionary = corpora.Dictionary(df['tokens'])
corpus = [dictionary.doc2bow(text) for text in df['tokens']]

# Build LDA model with 10 topics
lda_model = models.LdaModel(corpus=corpus,
                            id2word=dictionary,
                            num_topics=10,
                            random_state=42,
                            passes=10,
                            alpha='auto')

# Print topics
for idx, topic in lda_model.print_topics(-1):
    print(f"Topic {idx}: {topic}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 44.2 MB/s eta 0:00:00
Topic 0: 0.012*"company" + 0.011*"year" + 0.008*"finnish" + 0.007*"said" + 0.007*"contract" + 0.006*"signed" + 0.005*"agreement" + 0.005*"awarded" + 0.005*"finland" + 0.005*"result"
Topic 1: 0.030*"million" + 0.015*"eur" + 0.015*"said" + 0.013*"company" + 0.013*"finnish" + 0.013*"oyj" + 0.010*"percent" + 0.009*"hel" + 0.009*"net" + 0.009*"maker"
Topic 2: 0.015*"company" + 0.013*"new" + 0.012*"market" + 0.010*"said" + 0.007*"n" + 0.006*"finland" + 0.006*"paper" + 0.006*"pharmaceutical" + 0.006*"capacity" + 0.006*"finnish"
Topic 3: 0.099*"eur" + 0.055*"mn" + 0.033*"profit" + 0.031*"sale" + 0.030*"net" + 0.026*"period" + 0.025*"operating" + 0.017*"increased" + 0.017*"quarter" + 0.016*"company"
Topic 4: 0.008*"service" + 0.007*"said" + 0.007*"market" + 0.006*"new" + 0.006*"project" + 0.005*"nokia" + 0.004*"mobile" + 0.004*"paper" + 0.004*"group" + 0.004*"music"
Topic 5: 0.024*"mln" + 0.013*"euro" + 0.011*"finnis

**5. Coherence Score**

In [5]:
from gensim.models import CoherenceModel

coherence_model_lda = CoherenceModel(model=lda_model, texts=df['tokens'],
                                     dictionary=dictionary, coherence='c_v')
coherence_score = coherence_model_lda.get_coherence()
print("Coherence Score:", coherence_score)

Coherence Score: 0.35557873178592125


**6. Topic Visualization**

In [6]:
!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
vis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 29.6 MB/s eta 0:00:00


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.272253  0.009266       1        1  15.212385
1     -0.073225 -0.092440       2        1  12.196407
5     -0.003573 -0.078823       3        1  11.576030
7      0.064463 -0.038139       4        1  10.666389
8      0.056624 -0.019162       5        1   9.688858
0      0.051620 -0.023844       6        1   9.144028
9     -0.030304  0.163109       7        1   8.205074
6      0.058482 -0.008786       8        1   8.148298
2      0.058600  0.072724       9        1   7.728813
4      0.089567  0.016096      10        1   7.433717, topic_info=             Term        Freq       Total Category  logprob  loglift
93            eur  503.000000  503.000000  Default  30.0000  30.0000
95             mn  216.000000  216.000000  Default  29.0000  29.0000
63         profit  145.000000  145.000000  Default  28.0000  28.0000
61            net  152.000000  152.000000  Default  27.0000  27.0000
230       million  165.000000  165.000000  Default  26.0000  26.0000
...           ...         ...         ...      ...      ...      ...
1823  cooperation    5.209642   29.124042  Topic10  -5.8949   0.8781
301        baltic    4.828033   20.649417  Topic10  -5.9710   1.1459
266          unit    5.160597   35.538606  Topic10  -5.9044   0.6696
2         company    6.055761  312.558017  Topic10  -5.7444  -1.3446
156       finnish    5.443177  189.117824  Topic10  -5.8511  -0.9489

[738 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
4789      1  0.869043       abb
4227      6  0.824821     abloy
694       2  0.187773  activity
694       3  0.093886  activity
694       4  0.093886  activity
...     ...       ...       ...
750       7  0.164591       yit
750       8  0.164591       yit
750      10  0.329182       yit
3110      7  0.619924      york
2535     10  0.842006  zinclead

[1807 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 2, 6, 8, 9, 1, 10, 7, 3, 5])

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


**7. Dependency Parsing (Two Sentences)**

In [7]:
import spacy
from spacy import displacy
import random

nlp = spacy.load("en_core_web_sm")

# Select two random sentences with >=10 words
long_sentences = df[df['News Headline'].str.split().apply(len) >= 10]['News Headline']
samples = random.sample(list(long_sentences), 2)

for sent in samples:
    doc = nlp(sent)
    displacy.render(doc, style="dep", jupyter=True)

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

**8. Conclusion**
- We performed EDA and preprocessing.
- Extracted 10 topics using LDA.
- Computed coherence score to evaluate topic quality.
- Visualized topics with pyLDAvis.
- Parsed two long sentences with spaCy dependency parser.

